In [17]:
import pandas as pd
import numpy as np

In [18]:
names_df = pd.read_csv("raw_data/Names.csv",usecols = [0,1])
address_df = pd.read_csv("raw_data/address.csv").sample(100, random_state = 2022, ignore_index = True)
cardno_df = pd.read_csv("raw_data/cardno.csv",usecols = [0]).sample(100, random_state = 2022, ignore_index = True)
cardno_df

,Card_Number
0,2357-9896-6403-5970
1,6514-4444-5642-2615
2,4783-2852-8264-4966
3,4856-2191-9879-6388
4,4138-5166-7490-1188
...,...
95,7837-4036-5999-1672
96,3652-5528-4808-8216
97,7658-7645-4568-1461
98,4093-1634-8086-4601


### Customers data

**Names attribute**

In [19]:
customer_df = names_df.iloc[:100,:].copy()
customer_df.rename(columns = {'First Name': "fname", "Last Name": "lname"}, inplace = True)
customer_df

,fname,lname
0,Mechelle,Stoneman
1,Tyesha,Freitag
2,Dean,Stoecker
3,Annelle,Pickney
4,Margareta,Tacy
...,...,...
95,Glenda,Woolfolk
96,Alyson,Hosack
97,Francis,Sines
98,Riley,Bagnall


**id, age, email, address**

In [20]:
np.random.seed(2022)
email_suffix = ['@yahoo.com', '@gmail.com', '@hotmail.com', '@nyu.edu']
email_suffix_list = np.random.choice(email_suffix,100)

age = np.random.randint(18, 60, 100)
# print(age)

cid = np.random.randint(111111, 999999, 100)
# print(cid)

email_list = [f.lower()+l.lower()+e for f,l,e in zip(customer_df['fname'], customer_df['lname'], email_suffix_list)]
# print(email_list)

address_list = [', '.join(val) for val in address_df.astype(str).values.tolist()]
# print(address_list)


customer_df['customer_id'] = cid
customer_df['age'] = age
customer_df['email'] = email_list
customer_df['address'] = address_list

In [21]:
customer_df = customer_df[['customer_id', 'fname', 'lname', 'age', 'email', 'address']]

In [216]:
customer_df.to_csv('final_data/Customers.csv', index = False)

## Cards data

In [213]:
np.random.seed(2022)
customer_sample = customer_df.sample(100, replace = True, random_state = 2022)

expiry_month = np.random.randint(1,13, 100)
# print(expiry_month)

expiry_year = np.random.randint(2024,2030, 100)
# print(expiry_year)


cards_df = pd.DataFrame()
cards_df['card_number'] = [''.join(val.strip().split('-')) for val in cardno_df['Card_Number']]
cards_df['customer_id'] = list(customer_sample['customer_id'])
cards_df['billing_address'] = list(customer_sample['address'])
cards_df['expiry_month'] = expiry_month
cards_df['expiry_year'] = expiry_year


In [214]:
cards_df.to_csv('final_data/HasCards.csv', index = False)

## Sold by

In [3]:
product_df = pd.read_csv('final_data/products.csv')
seller_df = pd.read_csv('final_data/sellers.csv')

In [16]:
np.random.seed(2022)
seller_id = np.random.choice(seller_df['seller_id'],len(product_df['product_id']))
seller_id

soldby_df = pd.DataFrame({'seller_id':seller_id, 'product_id':product_df['product_id']})
soldby_df.to_csv('final_data/SoldBy.csv', index = False)

## Bought in

In [48]:
# import uuid
# uuids = [uuid.uuid4().hex for i in range(1000)]
# uuid_df = pd.DataFrame({'uuid':uuids})
# uuid_df.to_csv('final_data/uuid.csv', index = False)

In [92]:
uuid_df = pd.read_csv('final_data/uuid.csv')
product_df = pd.read_csv('final_data/products.csv')

In [114]:
np.random.seed(2022)
n_order = 40
gen_order_id = list(uuid_df['uuid'][:n_order].values)
n_items = np.random.randint(1, 11, size = n_order)

order_id = []
product_id = []
quantity = []

for _id, _ni in zip(gen_order_id, n_items):
    t_products = np.random.choice(product_df['product_id'], _ni, replace = False)
    t_quantity = np.random.randint(1, 50, size = _ni)
    for i in range(_ni):    
        order_id.append(_id)
        product_id.append(t_products[i])
        quantity.append(t_quantity[i])
        
boughtin_df = pd.DataFrame({'order_id':order_id, 'product_id':product_id, 'quantity':quantity})
boughtin_df.to_csv('final_data/boughtin.csv', index = False)

## Warehouse

In [337]:
from faker import Faker
fake = Faker()
Faker.seed(2022)
ware_name = [fake.company() for _ in range(25)]
ware_addr = [fake.street_address() for _ in range(25)]
ware_zip = [fake.postcode() for _ in range(25)]

warehouse_df = pd.DataFrame({'warehouse_name':ware_name, 'warehouse_location':ware_addr,'warehouse_zipcode':ware_zip})
warehouse_df.to_csv('final_data/warehouse.csv', index = False)

In [154]:
# np.random.seed(2022)
# n_order = 40
# gen_order_id = list(uuid_df['uuid'][:n_order].values)


# warehouse = [{'name':'C-bay Warehouse1', 'location': '3412-3416 Garfield Avenue Commerce, CA 90040', 'zipcode':90040}, 
#              {'name':'C-bay Warehouse2', 'location': '135 West 36th Street, New York, NY, 10018', 'zipcode':10018}]

# all_warehouses = np.random.choice(warehouse, size = len(gen_order_id))


# ware_name = [ent['name'] for ent in all_warehouses]
# ware_loc = [ent['location'] for ent in all_warehouses]
# ware_zip = [ent['zipcode'] for ent in all_warehouses]

# shippedfrom_df = pd.DataFrame({'order_id':gen_order_id, 'name':ware_name, 'location': ware_loc, 'zipcode':ware_zip})
# shippedfrom_df.to_csv('final_data/ShippedFromWarehouse.csv', index = False)

## ordered by

Create table OrderedByAppliedonShippedfrom(
	order_id varchar(120) primary key,
    customer_id varchar(60) not null,
    warehouse_name varchar(50) not null,
    warehouse_zipcode integer not null,
    coupon_id varchar(60),
	order_placed_on date not null,
	total_amount real not null,
	tax_amount real not null,
	delivery_charges real not null,
	billed_amount real not null,
	foreign key(coupon_id) references  Coupons(Coupon_id),
	foreign key (customer_id) references Customers(customer_id),
    foreign key(warehouse_name, warehouse_zipcode) references Warehouse(warehouse_name, warehouse_zipcode)
);

In [344]:
uuid_df = pd.read_csv('final_data/uuid.csv')
customer_df = pd.read_csv('final_data/Customers.csv')
coupon_df = pd.read_csv('final_data/coupons.csv')
warehouse_df = pd.read_csv('final_data/warehouse.csv')

In [355]:
np.random.seed(2022)
from faker import Faker
fake = Faker()
Faker.seed(2022)

n_order = 40
gen_order_id = list(uuid_df['uuid'][:n_order].values)


all_warehouses = np.random.choice(range(len(warehouse_df)), size = len(gen_order_id))

# ware_name = [ent['name'] for ent in all_warehouses]
# ware_loc = [ent['location'] for ent in all_warehouses]
# ware_zip = [ent['zipcode'] for ent in all_warehouses]



gen_customer_id = np.random.choice(customer_df['customer_id'], size = n_order)
gen_order_date = [fake.date_between(start_date='-2y', end_date='now').strftime('%m-%d-%Y') for i in range(40)]

order_coupon = np.random.choice(coupon_df['coupon_id'], size = n_order)

ORDER = uuid_df[:40]
or_bo = pd.merge(ORDER, boughtin_df, left_on = 'uuid', right_on = 'order_id')
or_bo_pro = pd.merge(or_bo, product_df, left_on = 'product_id', right_on = 'product_id')
or_bo_pro_df = or_bo_pro[['order_id','product_id', 'quantity', 'price']].copy()
or_bo_pro_df['total_amount'] = or_bo_pro_df['quantity'] * or_bo_pro_df['price']
order_total = or_bo_pro_df[['order_id','total_amount']].groupby('order_id').sum().reset_index()

order_total['customer_id'] = gen_customer_id
order_total['order_placed_on'] = gen_order_date
order_total['tax_amount'] = order_total['total_amount'] * 0.18
order_total['delivery_charges'] = order_total['total_amount'] * 0.05

billed_ammount = []
rand_coupon = []
ware_name = []
ware_zip = []
for c, t,w in zip(order_coupon, (order_total['total_amount'] + order_total['tax_amount'] + order_total['delivery_charges']), all_warehouses):
    ware_name.append(warehouse_df.loc[w,'warehouse_name'])
    ware_zip.append(warehouse_df.loc[w,'warehouse_zipcode'])
    
    if np.random.choice([True, False]):
        rand_coupon.append(c)
        dis = coupon_df[coupon_df['coupon_id'] == c]['discount'].values[0]
        billed = t - t*dis
        billed_ammount.append(billed)
        
    else:
        rand_coupon.append('')
        billed_ammount.append(t)
    

order_total['coupon_id'] = rand_coupon
order_total['billed_amount'] = billed_ammount
order_total['warehouse_name'] = ware_name
order_total['warehouse_zipcode'] = ware_zip

orderedby_df = order_total[['order_id', 'customer_id','warehouse_name','warehouse_zipcode','coupon_id','order_placed_on', 'total_amount', 'tax_amount','delivery_charges', 'billed_amount']]
orderedby_df.to_csv('final_data/OrderedByAppliedonShippedfrom.csv', index = False)
orderedby_df

,order_id,customer_id,warehouse_name,warehouse_zipcode,coupon_id,order_placed_on,total_amount,tax_amount,delivery_charges,billed_amount
0,03bce229db954c29a1778e19b3f23a17,807336,Reynolds Inc,9524,,01-29-2022,2880.59,518.5062,144.0295,3543.125700
1,0522eb53333e40ef82364616b3eb3b6e,884910,Johnston Inc,64683,HWTIUT,07-25-2021,1173.34,211.2012,58.6670,1313.319462
2,05ca9acd7a4f477cae0bbec684920132,539977,Ross-Mercer,62170,MAILAP,11-21-2021,4998.66,899.7588,249.9330,4918.681440
3,0a0ebf9b43d94579b9827771b4f9b0b2,610746,Ross-Mercer,62170,,02-09-2022,2138.75,384.9750,106.9375,2630.662500
4,158f669c88604ae7a9d208ab67842c77,510028,Johnston Inc,64683,GTFWGT,08-10-2021,356.79,64.2222,17.8395,373.023945
5,195c7bfcbd6c4750969d72a6386a8ea9,968675,"Brown, Humphrey and Callahan",8592,,03-13-2022,16685.93,3003.4674,834.2965,20523.693900
6,1bf6dd1bac97481ea8baba0b7c078198,299649,"Curry, Cox and Watson",87890,LERDKB,01-29-2021,679.60,122.3280,33.9800,693.803640
7,1e13999c8edf420fabc778fdac811086,155199,Strickland-Martinez,78049,,01-19-2022,1714.51,308.6118,85.7255,2108.847300
8,24f674365ebd4c79a58313ea33b205cf,235264,"Curry, Cox and Watson",87890,,08-13-2022,7923.81,1426.2858,396.1905,9746.286300
9,2dbc29088f7c49f7806cf348e102cdde,685032,Johnston Inc,64683,NHTBVT,06-03-2022,8918.23,1605.2814,445.9115,8994.926778


In [275]:

coupon_df[coupon_df['coupon_id'] == 'WAFANK']['discount'].values[0]

0.08

In [276]:
3542 - 3542*0.08

3258.64